In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
import random

file_path = '../../datasets/patient-doctor.txt'

In [2]:
def read_conversations_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversations = file.readlines()
    return conversations

In [3]:
body_texts = read_conversations_from_file(file_path)
body_texts

['Patient: Hello, Good morning doctor.\n',
 'Doctor: Good morning, how are you feeling today?\n',
 "Patient: I've been feeling quite anxious lately, it's been hard to relax.\n",
 'Doctor: I see. Can you tell me more about what might be causing this anxiety?\n',
 "Patient: I think it's mainly related to my job and the pressure I'm under.\n",
 "Doctor: Stress at work can definitely take a toll on our well-being. Tell me about your job and the specific challenges you're facing.\n",
 "Patient: I work in a highly demanding environment, and I constantly feel like I'm not meeting expectations.\n",
 'Doctor: That sounds tough. Do you have any support or coping mechanisms in place when you feel overwhelmed?\n',
 "Patient: I try to talk to my friends, but they don't always understand what I'm going through.\n",
 "Doctor: Having a strong support system is important. Let's explore some other ways to manage stress effectively.\n",
 "Patient: I've also been having trouble sleeping. My mind just won'

## Text Cleaning

In [4]:
# Generate a n sized randomly selected chunk from a series
random_string = lambda a,n: ' '.join(a.sample(n, replace=False))
# Gives the vocabulary from a string.
vocab = lambda s: sorted(list(set(s)))
# Cleans a string so it only has english chars and white spaces. 
pattern = r'[^A-Za-z\s]'
clean_str = lambda s: re.sub(pattern, '', s).lower()
# Removes a specific set of characters from a string.
remove_chars = lambda t,c: re.compile('|'.join(map(re.escape, c))).sub(' ', t)
# Modify a string into n sized chunks
chunkify = lambda s,n: [s[i:i+4].strip() for i in range(0, len(s), n)]

In [5]:
## INITIAL TEXT DATA
body_texts = "".join(body_texts)
print(vocab(body_texts))

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—']


In [6]:
## DOING REFERENCE CLEANING
body_texts = clean_str(body_texts)
print(vocab(body_texts))

['\n', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [7]:
## DOING REFERENCE CLEANING
body_texts = remove_chars(body_texts,['\xa0','\u2002','\u2005','\u200a','\t','\n','\r'])
print(vocab(body_texts))

[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [8]:
# body_texts = [i for i in body_texts.split() if len(i) < 20]
# print(body_texts)

In [9]:
vocab_size = len(vocab(body_texts))
vocab_size

27

In [10]:
# data = {}
# for i in set(body_texts):
#     data[i] = body_texts.count(i)
    
# # Sort the dictionary by values in descending order and select the top 20 items
# sorted_data = sorted(data.items(), key=lambda x: x[1], reverse=True)[:20]

# # Extract the keys and values from the sorted data
# keys = [item[0] for item in sorted_data]
# values = [item[1] for item in sorted_data]

# # Create a bar chart
# plt.bar(keys, values)

# # Add labels and title
# plt.xlabel('Words')
# plt.ylabel('Occurrences')
# plt.title('Top 20 Words')

# # Rotate x-axis labels for better readability if needed
# plt.xticks(rotation=45)

# # Display the chart
# plt.show()

## Training Model

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [12]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 16 # what is the maximum context length for predictions?
hidden_size = 8
max_iters = 10000
eval_interval = 100
learning_rate = 0.01
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 1000

In [13]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(vocab(body_texts)) }
itos = { i:ch for i,ch in enumerate(vocab(body_texts)) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ' '.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [14]:
# Train and test splits
data = torch.tensor(encode(body_texts), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [15]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)

    def forward(self, inputs):
        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)

        attn_weights = torch.matmul(query, key.transpose(-2, -1))
        attn_weights = F.softmax(attn_weights, dim=-1)
        attended = torch.matmul(attn_weights, value)

        return attended

In [17]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [18]:
class BigramLanguageModelAttention(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.token_embedding_table = nn.Embedding(vocab_size, hidden_size)
        self.self_attention = SelfAttention(hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)
    def forward(self, idx, targets=None):
        embeddings = self.token_embedding_table(idx)  # (B, T, hidden_size)
        attended = self.self_attention(embeddings)  # (B, T, hidden_size)
        logits = self.fc(attended)  # (B, T, vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, _ = logits.shape
            logits = logits.view(B * T, -1)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            last_logits = logits[:, -1, :]  # (B, vocab_size)
            probs = F.softmax(last_logits, dim=-1)  # (B, vocab_size)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx
    def compute_reward(self, idx, targets):
        _, loss = self(idx, targets)
        reward = loss.item()  # Negative loss as the reward
        return reward

In [19]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [34]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.4200, val loss 3.4094
step 1: train loss 3.3125, val loss 3.3462
step 2: train loss 3.1635, val loss 3.1562
step 3: train loss 3.0884, val loss 3.0829
step 4: train loss 2.9966, val loss 2.9957


KeyboardInterrupt: 

In [24]:
torch.save(m, 'PD_generative.pth')

In [30]:
import time
context = torch.zeros((1, 1), dtype=torch.long, device=device)
max_new_chars = 200
max_new_words = 20

user_input = clean_str(input("Context Generator, Give context: "))
# user_encoded = encode(user_input) # for chars
user_encoded = encode(user_input) # for words
tensor = torch.tensor(user_encoded, dtype=torch.long, device=device).unsqueeze(0)

generated_text = ""
print("Generated Text:")

def clean_text(text):
    # Convert to lowercase if the value is a string
    if isinstance(text, str):
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Remove digits and extra spaces
        text = re.sub(r'\d', '', text)
        text = re.sub(r'\s+', ' ', text)

        return text.strip()
    else:
        return text
    
# generate from chars
for _ in range(max_new_chars):
    res = m.generate(tensor, max_new_tokens=1)[0].tolist()
    context = torch.cat((tensor, torch.tensor(res[-1:], dtype=torch.long, device=device).unsqueeze(0)), dim=1)
    generated_word = decode(res[-1:])
    time.sleep(0.01)
    generated_text += generated_word
    if generated_text[-1] == ' ':
        print(clean_text(generated_text[:-1]), end=' ')
        generated_text = ''
        
# # generate from words
# for _ in range(max_new_words):
#     res = m.generate(context, max_new_tokens=1)[0].tolist()
#     context = torch.cat((context, torch.tensor([res[-1]], dtype=torch.long, device=device).unsqueeze(0)), dim=1)
#     generated_word = decode(res[-1:])
#     time.sleep(0.01)
#     print(generated_word, end=' ')

Context Generator, Give context: Hello, I am feeling anxious
Generated Text:
t  u   tttit e  eua  i    i s oe i   t tu tos e tscm sk ie e    stee e utst  ttt  s  ea  e o iuatotut f ee e eu e tuee o eh o   o o  e  s ei   iots  e  es ost     etiusppu   ioisse ts     i  o p se   

In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 500
eval_interval = 1
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

10.710555 M parameters
